In [1]:
import tweepy  # https://github.com/tweepy/tweepy
import csv
screen_name="EdinburghUni"
# Twitter API credentials
consumer_key = "tndAcVLSnfdGVxHXSrtCx7yY0"
consumer_secret = "dXZD2WAHW6vfQIo1L77HLGp9SmvETn058sMaXaQoh9Zm7HcQD3"
access_key = "1405286036473843714-QVmhQZVHpSPzICwqjXokxVoSTRslb5"
access_secret = "7gBfupUaKCM5boYQYJK1YYDEvpILhTTIWr3ESXGnS7gOQ"

auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)


In [3]:
# initialize a list to hold all the tweepy Tweets
alltweets = []

# make initial request for most recent tweets (200 is the maximum allowed count)
new_tweets = api.user_timeline(screen_name=screen_name, count=200,tweet_mode="extended")
alltweets.extend(new_tweets)

# save the id of the oldest tweet less one
oldest = alltweets[-1].id - 1
print(len(alltweets))

200


In [4]:

# You can only get a maximum of 200 tweets in one request. 
# However, you can make successive requests for older tweets. The maximum number of tweets that you can get in a timeline is 3200.
while len(new_tweets) > 0:
    
    # all subsiquent requests use the max_id param to prevent duplicates
    new_tweets = api.user_timeline(screen_name=screen_name, count=200, max_id=oldest,tweet_mode="extended")

    # save most recent tweets
    alltweets.extend(new_tweets)

    # update the id of the oldest tweet less one
    oldest = alltweets[-1].id -1
print(len(alltweets))

3202


In [5]:
#extract the place name from the timeline
placeset=set()
for tweet in alltweets :
    if tweet.place is not None and tweet.place.place_type=="city":
      # print(tweet.place.place_type)
      placeset.add(tweet.place.name)
placeslist=list(placeset)
print(placeslist)

['Edinburgh', 'Berlin']


In [54]:
from nltk.probability import FreqDist
hashtags=[]
for tweet in alltweets:
  hashtaginfo=tweet.entities.get("hashtags")
  if len(hashtaginfo)!=0:
    hashtags.append(hashtaginfo[0].get("text").lower())
# print(hashtags)
tag_freq = FreqDist(hashtags)
print(tag_freq.most_common())


<bound method Counter.most_common of FreqDist({'edinburghgrad': 92, 'covid19': 53, 'edinburghimpact': 35, 'jamestaitblack': 33, 'sharingthingspodcast': 29, 'edwelcome': 26, 'edopenday': 22, 'doorsopenday': 20, 'edinburgh': 19, 'livinggratefully': 18, ...})>


In [66]:
# divide tags into first class and second class
tag2class=[tag[0] for tag in tag_freq.most_common() if tag[1]>=5] # high possibility
tag1class=[tag[0] for tag in tag2class if tag[1]>=10] # normal possibility
print(tag1class)
print(tag2class)

['edinburghgrad', 'covid19', 'edinburghimpact', 'jamestaitblack', 'sharingthingspodcast', 'edwelcome', 'edopenday', 'doorsopenday', 'edinburgh', 'livinggratefully', 'internationalwomensday', 'edscifest', 'edinburghfuturesconversations', 'eduniedscifest', 'drawntoedinburgh', 'worldheritageday', 'womenshistorymonth']
['edinburghgrad', 'covid19', 'edinburghimpact', 'jamestaitblack', 'sharingthingspodcast', 'edwelcome', 'edopenday', 'doorsopenday', 'edinburgh', 'livinggratefully', 'internationalwomensday', 'edscifest', 'edinburghfuturesconversations', 'eduniedscifest', 'drawntoedinburgh', 'worldheritageday', 'womenshistorymonth', 'europeday', 'studentvolunteeringweek', 'womeninscience', '1daywithoutus', 'gbbo', 'internationaldayofwomenandgirlsinscience', 'edinburghseven', 'clearing', 'scotappweek20', 'gew2019', 'africaweek', 'fcl18', 'universitychallenge', 'wideningparticipation', 'volunteersweek', 'nhs', 'internationalstudentsday', 'sv19', 'giffordlecture', 'worldoceansday', 'cancer', 'wo

In [7]:
outtweets = [tweet.full_text for tweet in alltweets]
# print(outtweets)
del(outtweets[0])
# print(outtweets)

In [8]:
import re
import snowballstemmer
#  remove stop words
stopwords = []
with open("stopwords.txt", "r") as f1:
    for stop_word in f1.readlines():
        if stop_word is not None:  # read stop words from a txt file and store then into a list.
            stopwords.append(stop_word.strip("\n"))


# preprocess used for builidng inverted index including tokenisation, casefolding, removing stop words and porter stemming.
def preprocess(text):
    remove_url = re.sub(r"http\S+", "", text)
    # remove_number=remove_url.replace('\d+', '')
    regEx = re.compile("\W").split(remove_url)  # split non-letter characters and store them into a list.
    token = [i for i in regEx if i != '']
    casefolding = [s.lower() for s in token if isinstance(s, str) == True]  # all in lower case.
    stopwords_removed = [w for w in casefolding if not w in stopwords]  # removing stop words.
    stemmer = snowballstemmer.stemmer('english')  # Porter stemmer by using snowball stemmer lib.
    return stemmer.stemWords(stopwords_removed)

preprocessed_tweets=[]
for tweet in outtweets:
  preprocessed_tweets.append(preprocess(tweet))
# print(preprocessed_tweets)

In [9]:
#word frequency
flat_words = [item for sublist in preprocessed_tweets for item in sublist]
# print(flat_words)

word_freq = FreqDist(flat_words)
print(word_freq.most_common(10))

from gensim.corpora import Dictionary

#create dictionary
text_dict = Dictionary(preprocessed_tweets)

#view integer mappings
# print(text_dict.token2id)

tweets_bow = [text_dict.doc2bow(tweet) for tweet in preprocessed_tweets]
# print(tweets_bow)

from gensim.models.ldamodel import LdaModel
#fitting lda model
tweets_lda = LdaModel(tweets_bow,
                      num_topics = 10,
                      id2word = text_dict,
                      random_state=1,
                      # passes=10
                     )

x=tweets_lda.show_topics()
alltopicWords=[]
for topicNum,topicWords in x:
  words_foreachtopic = re.findall("[a-zA-Z]+",x[topicNum][1])
  alltopicWords+=words_foreachtopic
alltopicWordsset=set(alltopicWords)
alltopicWords=list(alltopicWordsset)
print(alltopicWords)

[('amp', 905), ('student', 738), ('edinburghuni', 479), ('edinburgh', 478), ('rt', 461), ('univers', 446), ('find', 432), ('research', 424), ('studi', 327), ('year', 299)]
['student', 'find', 'uk', 'world', 'ceremoni', 'school', 'talbotrice', 'live', 'join', 'exhibit', 'work', 'eca', 'scientist', 'peopl', 'edinburgh', 'year', 'contact', 'congratul', 'challeng', 'part', 'edinburghuni', 'research', 'geosciences', 'edinburghgrad', 'suggest', 'graduat', 'life', 'award', 'studi', 'today', 'museum', 'book', 'open', 'amp', 'day', 'univers', 'lectur', 'team', 'rt', 'event', 'develop', 'offer']


In [10]:

newtweet="Berlin"
# !pip install geotext
# Determine if a city name exists because this python library only supports city name recognition but the tweet place entity can have place type other than city such as admin 
from geotext import GeoText
# 1. recognize the city name in the new tweet
placename=GeoText(newtweet).cities
if len(placename)!=0:
  if set(placename)<=set(placeslist):
    print("This new tweet is fine to post !")

 

This new tweet is fine to post !


In [67]:
# 2. Extract the hashtag words
# set a threshold (word frequency) rare or not 

# function to print all the hashtags in a text
def extract_hashtags(text):
    # initializing hashtag_list variable
    hashtag_list = []

    # splitting the text into words
    for word in text.lower().split():

        # checking the first charcter of every word
        if word[0] == '#':
            if word[1:]!="":
              # adding the word to the hashtag_list
              hashtag_list.append(word[1:].lower())
    # If all the hashtags of new tweet has appeared in previous tweets, then we decide to pass it
    
        
    return hashtag_list
    # printing the hashtag_list
    # print("The hashtags in \"" + text + "\" are :")
    # for hashtag in hashtag_list:
    #     print(hashtag)

In [71]:
text2 = "This day is beautiful ! #edinburghgrad #forest # cute"
if set(extract_hashtags(text2))<=set(tag2class):
  print("This tweet is fine to post!")

This tweet is fine to post!


In [13]:
# 3. Determine if the number of key tokens in new tweets is over the threshold
testtweet="Banning in-car smoking when children are present can reduce their exposure to tobacco smoke by more than 30 per cent, new research from @EdinUniUsher shows."
txt=preprocess(testtweet)
counter=0
for token in txt:
  if token in alltopicWords :
    counter+=1
    print(token) 
if counter>=round(len(alltopicWords)/10):
  print(counter)
  print("This tweet is fine to post!")

research


In [23]:
# 4.using lsi to measure the similarity of new tweet and old corpus
#if there are previous tweets that get over 0.97 score, then we can say the new tweet is similar to some previous tweets 
corpus=tweets_bow
id2word=text_dict
dictionary=text_dict
from gensim import models
lsi = models.LsiModel(corpus=tweets_bow, id2word=text_dict, num_topics=10)

doc = 'We are aware of the issues some graduates are having trying to access the booking system. We are looking into this and are trying to resolve this as soon as possible, We apologise for the inconvenience.'
# doc='Booking is now open!'
vec_bow = dictionary.doc2bow(preprocess(doc))
vec_lsi = lsi[vec_bow]  # convert the query to LSI space
# print(vec_lsi)

from gensim import similarities
index = similarities.MatrixSimilarity(lsi[corpus])  # transform corpus to LSI space and index it

index.save('deerwester.index')
index = similarities.MatrixSimilarity.load('deerwester.index')

sims = index[vec_lsi]  # perform a similarity query against the corpus
# print(list(enumerate(sims)))  # print (document_number, document_similarity) 2-tuples

sims = sorted(enumerate(sims), key=lambda item: -item[1])
for doc_position, doc_score in sims:
    if doc_score>=0.97:
      print(doc_score, outtweets[doc_position])

1.0 We are aware of the issues some graduates are having trying to access the booking system. We are looking into this and are trying to resolve this as soon as possible, We apologise for the inconvenience.
